## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Riyadh,SA,24.6877,46.7219,87.85,11,0,9.22,dust
1,1,Nizhnevartovsk,RU,60.9344,76.5531,68.02,64,0,6.71,clear sky
2,2,Albany,US,42.6001,-73.9662,68.92,75,80,6.42,broken clouds
3,3,Coihaique,CL,-45.5752,-72.0662,37.35,65,0,3.44,clear sky
4,4,Anadyr,RU,64.7500,177.4833,50.50,57,75,13.42,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# we convert the string input into a float
min_temp = float(input("What is the minimum temperature (°F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (°F) you would like for your trip? "))

What is the minimum temperature (°F) you would like for your trip? 69
What is the maximum temperature (°F) you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_city = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
prefered_city.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Pokhara,NP,28.2333,83.9833,71.74,88,100,4.12,overcast clouds
13,13,Rikitea,PF,-23.1203,-134.9692,75.38,74,100,13.65,overcast clouds
22,22,Buta,CD,2.7858,24.7300,69.62,98,100,2.06,light rain
26,26,Shahrud,IR,36.4182,54.9763,71.24,54,0,10.29,clear sky
27,27,Biak,ID,-0.9131,122.8766,77.04,86,100,4.36,overcast clouds
29,29,Airai,TL,-8.9266,125.4092,73.00,47,66,3.40,broken clouds
37,37,Yellowknife,CA,62.4560,-114.3525,70.32,52,40,14.97,scattered clouds
40,40,Ponta Do Sol,PT,32.6667,-17.1000,70.43,69,87,5.48,overcast clouds
41,41,Acarau,BR,-2.8856,-40.1200,78.10,83,5,9.86,clear sky
42,42,Itoman,JP,26.1247,127.6694,79.57,92,75,11.50,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
prefered_city.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_city_df = prefered_city.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Pokhara,NP,71.74,overcast clouds,28.2333,83.9833,
13,Rikitea,PF,75.38,overcast clouds,-23.1203,-134.9692,
22,Buta,CD,69.62,light rain,2.7858,24.7300,
26,Shahrud,IR,71.24,clear sky,36.4182,54.9763,
27,Biak,ID,77.04,overcast clouds,-0.9131,122.8766,
29,Airai,TL,73.00,broken clouds,-8.9266,125.4092,
37,Yellowknife,CA,70.32,scattered clouds,62.4560,-114.3525,
40,Ponta Do Sol,PT,70.43,overcast clouds,32.6667,-17.1000,
41,Acarau,BR,78.10,clear sky,-2.8856,-40.1200,
42,Itoman,JP,79.57,moderate rain,26.1247,127.6694,


In [8]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.NaN
        print("Hotel not found... skipping.")
        
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# check hotel df to see if hotel was scrapped
hotel_df.tail(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
666,Bahia Honda,CU,76.91,overcast clouds,22.9036,-83.1592,Casa Gualberto - Deysi
667,Mawlaik,MM,79.20,overcast clouds,23.6333,94.4167,AKZ Guest House
672,Hami,CN,73.56,light rain,42.8000,93.4500,Laoyutang Resort
678,Tura,IN,79.95,light rain,25.5198,90.2201,Hotel RIKMAN Continental
681,Filadelfia,US,72.00,overcast clouds,39.9523,-75.1638,"The Ritz-Carlton, Philadelphia"
693,Progreso,MX,75.45,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
694,Sungaipenuh,ID,74.34,scattered clouds,-2.0833,101.3833,Hotel Jaya Wisata 2
695,Jaru,BR,75.88,broken clouds,-10.4389,-62.4664,Hotel Paraná
703,Gushikawa,JP,78.80,moderate rain,26.3544,127.8686,Kirari Guest House
707,Shuangcheng,CN,76.68,light rain,45.3500,126.2833,Xingcheng Hotel


In [11]:
# sees if missing values (NaN)
hotel_df.count()

City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             184
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Pokhara,NP,71.74,overcast clouds,28.2333,83.9833,Butterfly Lodge
13,Rikitea,PF,75.38,overcast clouds,-23.1203,-134.9692,People ThankYou
22,Buta,CD,69.62,light rain,2.7858,24.7300,HOTEL BISO NA BISO
26,Shahrud,IR,71.24,clear sky,36.4182,54.9763,Royal Hotel
27,Biak,ID,77.04,overcast clouds,-0.9131,122.8766,Penginapan Kim
...,...,...,...,...,...,...,...
707,Shuangcheng,CN,76.68,light rain,45.3500,126.2833,Xingcheng Hotel
709,Jinchang,CN,70.70,clear sky,38.4953,102.1739,Mingshihui Jinchang International Hotel
711,Takoradi,GH,78.93,light rain,4.8845,-1.7554,Raybow International Hotel
712,Qujing,CN,69.15,overcast clouds,25.4833,103.7833,Yulong Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))